In [ ]:
from skopt import BayesSearchCV
from sklearn.model_selection import StratifiedKFold
import numpy as np
import matplotlib.pyplot as plt

class BayesianOptimization:
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def custom_method(self, param1, param2):
        qtd_total_movimentada = self.X[:, 0]
        ratio = self.X[:, 1]

        result = np.zeros(len(self.X), dtype=int)
        result[qtd_total_movimentada >= param1] += 1
        result[ratio >= param2] += 1

        return result

    def custom_metric(self, y_true, y_pred):
        tp = np.sum((y_true == 1) & (y_pred == 1))
        fp = np.sum((y_true == 0) & (y_pred == 1))
        return tp / (tp + fp)

    def objective_function(self, params):
        params['param1'] = int(params['param1'])
        params['param2'] = int(params['param2'])

        y_pred = self.custom_method(params['param1'], params['param2'])
        score = self.custom_metric(self.y, y_pred)

        return score

    def optimize_parameters(self):
        param_space = {'param1': (1, 10), 'param2': (0, 1)}

        opt = BayesSearchCV(None, param_space, n_iter=50, n_jobs=-1, scoring=self.custom_metric, cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42))
        opt.fit(self.X, self.y)

        return opt

    def visualize_results(self, opt):
        results = opt.cv_results_
        plt.scatter(results['param_param1'], results['param_param2'], c=results['mean_test_score'], cmap='viridis', s=50)
        plt.colorbar(label='Mean of the custom metric')
        plt.xlabel('param1')
        plt.ylabel('param2')
        plt.title('Bayesian Optimization - Top 5 combinations')
        plt.show()

# Example usage:
X_train = np.array([[10, 0.5], [15, 0.8], [5, 0.3]])
y_train = np.array([1, 0, 1])

optimizer = BayesianOptimization(X_train, y_train)
optimal_params = optimizer.optimize_parameters()

print("Top 5 combinations:")
for i, params in enumerate(optimal_params.cv_results_['params']):
    if i >= 5:
        break
    print(f"Combination {i+1}: {params}")

optimizer.visualize_results(optimal_params)